In [58]:
import numpy as np
from astropy.io import ascii
import matplotlib.pyplot as plt
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.coordinates import Distance
from astropy import units
from astropy import units as u
from astropy.table import unique
from astropy import table
from astropy.table import Table
import os
from astropy.io import fits
from astropy.cosmology import WMAP9 as cosmo
from scipy.ndimage import gaussian_filter
import scipy.linalg as linalg
import math
from scipy.interpolate import interp1d
import pandas as pd
import cobra
from cobra.utils import dataset_io as ds
import glob
import flimflam as ff
import scipy.spatial as sp
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import plot

In [21]:
info = pd.read_table('/Users/huangyuxin/Desktop/flimflam_config/FRB_list/FF_fields.txt', sep='\s+')
cobra_frbs = info[0:9].reset_index(drop=True)

In [22]:
targlist = []
for i in range(9):
    targlist.append(pd.read_csv(f"target_list/{cobra_frbs.loc[i,'FRB']}_targ.csv"))

In [23]:
# Copy the code in flimflam_config/Reconstruction_Test_for_Keck
def spherical2cartesian(ra,dec,redshift):
    #output is in Mpc/h
    distance = cosmo.comoving_distance(redshift)
    coords_spherical = coord.SkyCoord(ra*u.degree,dec*u.degree,distance=distance)
    coords_cartesian = coords_spherical.cartesian
    return (coords_cartesian.x.value*cosmo.h,coords_cartesian.y.value*cosmo.h,coords_cartesian.z.value*cosmo.h)
def cartesian2spherical(x,y,z):
    #input must be in Mpc/h
    coords_cartesian = coord.CartesianRepresentation(x*u.Mpc,y*u.Mpc,z*u.Mpc)
    coords_spherical = coord.SkyCoord(coords_cartesian.represent_as(coord.SphericalRepresentation))
    #ra= ((coords_spherical.ra.value-180)%360-180)*u.deg #optional
    def z_from_dist(distance):
      dummyred = np.linspace(0.,10.,10000)
      dummydist = cosmo.comoving_distance(dummyred)
      res = np.interp(distance,dummydist,dummyred)
      return (res)
    redshift = z_from_dist(coords_spherical.distance)
    return (coords_spherical.ra.value,coords_spherical.dec.value,redshift)

In [24]:
idx=3
ob = (targlist[idx]['FLIMFLAM_observed'])
redshift = targlist[idx].loc[ob,'z'].to_numpy()
redshift[redshift<0]=0
x,y,z = spherical2cartesian(targlist[idx].loc[ob,'ra'].to_numpy(), targlist[idx].loc[ob,'dec'].to_numpy(), redshift)

In [25]:
# FRB Cartesian coordinate
pos_frb = SkyCoord(cobra_frbs.loc[idx,'ra'], cobra_frbs.loc[idx,'dec']
         , distance=cosmo.comoving_distance(cobra_frbs.loc[idx,'z']), frame='icrs')
x_frb = pos_frb.cartesian.x.value*cosmo.h
y_frb = pos_frb.cartesian.y.value*cosmo.h
z_frb = pos_frb.cartesian.z.value*cosmo.h

In [26]:
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

#旋转矩阵 欧拉角
def rotate_mat(axis, radian):
    rot_matrix = linalg.expm(np.cross(np.eye(3), axis / linalg.norm(axis) * radian))
    return rot_matrix

# 分别是x,y和z轴,也可以自定义旋转轴
axis_x, axis_y, axis_z = [1,0,0], [0,1,0], [0,0,1]
rand_axis = unit_vector(np.cross([x_frb,y_frb,z_frb],[1,0,0]))

#旋转角度
yaw = angle_between([x_frb,y_frb,z_frb],[1,0,0])

#返回旋转矩阵
rot_matrix = rotate_mat(rand_axis, yaw)

# 计算点绕着轴运动后的点
x_rot,y_rot,z_rot = np.dot(rot_matrix, np.array([x,y,z]))
x_rotf,y_rotf,z_rotf = np.dot(rot_matrix, np.array([x_frb,y_frb,z_frb]))

In [59]:
# Create the interactive 3D scatter plot using plotly
trace = go.Scatter3d(x=x_rot, y=y_rot, z=z_rot, mode='markers', marker=dict(size=3, opacity=0.5), name='Observed galaxies')
trace_frb = go.Scatter3d(x=[x_rotf], y=[0], z=[0], mode='markers',
    marker=dict(size=6, color='red', symbol='diamond'), name='FRB20191001A'
)

layout = go.Layout(
    scene=dict(
        xaxis=dict(nticks=10, range=[0,x_rotf+50], backgroundcolor="rgba(0, 0, 0,0)",
                                         gridcolor="white",
                                         showbackground=True,
                                         zerolinecolor="black", linecolor="black"),
        yaxis=dict(nticks=10, range=[-15,15], backgroundcolor="rgba(0, 0, 0,0)",
                                         gridcolor="white",
                                         showbackground=True,
                                         zerolinecolor="black", linecolor="black"),
        zaxis=dict(nticks=10, range=[-15,15], backgroundcolor="rgba(0, 0, 0,0)",
                                         gridcolor="white",
                                         showbackground=True,
                                         zerolinecolor="black", linecolor="black"),
        aspectratio=dict(x=15, y=5, z=5),
        bgcolor='white'  # Set the scene background color to transparent
    )
)

fig = go.Figure(data=[trace,trace_frb], layout=layout)

fig.show()

plot(fig, filename='FRB20191001A_lc.html')

'FRB20191001A_lc.html'